In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
california_df = pd.read_csv('datasets/cal_housing_clean.csv')

In [3]:
california_df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [4]:
x_data = california_df.drop(['medianHouseValue'], axis=1)

In [5]:
y_val = california_df['medianHouseValue']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_val, test_size=0.3,
                                                   random_state=101)

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()

In [9]:
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [10]:
x_train = pd.DataFrame(data=scaler.transform(x_train), columns=x_train.columns,index=x_train.index)

In [11]:
x_test = pd.DataFrame(data=scaler.transform(x_test), columns=x_test.columns, index=x_test.index)

In [12]:
california_df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [13]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [14]:
feat_cols = [age, rooms, bedrooms, population, households, income]

In [15]:
input_function = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [16]:
estimator = tf.estimator.DNNRegressor(feature_columns=feat_cols, hidden_units=[6,6,6])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efd0a663fd0>, '_service': None, '_save_checkpoints_steps': None, '_session_config': None, '_tf_random_seed': None, '_evaluation_master': '', '_model_dir': '/tmp/tmppx3_3jpi', '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_master': '', '_task_type': 'worker', '_is_chief': True, '_task_id': 0, '_num_ps_replicas': 0, '_train_distribute': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_save_summary_steps': 100}


In [25]:
estimator.train(input_fn=input_function,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppx3_3jpi/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into /tmp/tmppx3_3jpi/model.ckpt.
INFO:tensorflow:step = 1001, loss = 3.53204e+11
INFO:tensorflow:global_step/sec: 458.907
INFO:tensorflow:step = 1101, loss = 3.50537e+11 (0.223 sec)
INFO:tensorflow:global_step/sec: 528.084
INFO:tensorflow:step = 1201, loss = 3.04374e+11 (0.188 sec)
INFO:tensorflow:global_step/sec: 520.018
INFO:tensorflow:step = 1301, loss = 2.5101e+11 (0.193 sec)
INFO:tensorflow:global_step/sec: 472.824
INFO:tensorflow:step = 1401, loss = 3.5132e+11 (0.211 sec)
INFO:tensorflow:global_step/sec: 480.089
INFO:tensorflow:step = 1501, loss = 2.19583e+11 (0.209 sec)
INFO:tensorflow:global_step/sec: 491.672
INFO:tensorflow:st

In [26]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=10,num_epochs=1,shuffle=False)

In [27]:
pred_res = estimator.predict(pred_input_func)

In [28]:
predictions = list(pred_res)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppx3_3jpi/model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [29]:
vals = []
for i in predictions:
    vals.append(i['predictions'])

In [30]:
from sklearn.metrics import mean_squared_error

In [31]:
print(mean_squared_error(y_test,vals)**0.5)

102796.074879
